In [1]:
import random

import random
import math

num_ants = 10
alpha = 1.0
beta = 2.0
evaporation = 0.1
exploration = 0.9
num_iterations = 100


In [2]:
def generate_cities(num_cities=5, pheromone_initial=1, min_distance= 0, max_distance=100):
    cities = []
    for i in range(num_cities):
        x = random.randint(min_distance, max_distance)
        y = random.randint(min_distance, max_distance)
        cities.append((x, y))
     
    pheromone = [[pheromone_initial for j in range(num_cities)] for i in range(num_cities)]

    distances = []
    for i in range(num_cities):
        row = []
        for j in range(num_cities):
            if i == j:
                row.append(0)
            else:
                xi, yi = cities[i]
                xj, yj = cities[j]
                distance =int(math.sqrt((xi - xj)**2 + (yi - yj)**2))
                row.append(distance)
        distances.append(row)
        
    return distances, pheromone, cities



In [3]:
class Ant:
    def __init__(self, start_city, distances, cities):
        self.start_city = start_city
        self.v_cities = [start_city]
        self.total_distance = 0.0
        self.distances = distances
        self.cities =cities

    def select_next_city(self, pheromone):
        probs = []
        c_city = self.v_cities[-1] 
          #select current city by selecting the last city of visited
        total_pheromone = 0.0
        for city in range(len(self.distances)):
            if city not in self.v_cities:
                pheromone_level = pheromone[c_city][city]
                distance = self.distances[c_city][city]
                probability = (pheromone_level ** alpha) * ((1.0 / distance) ** beta)
                probs.append((city, probability))
                total_pheromone += probability
                #(T^alpha*(1/d)^beta)/(Sum(T^alpha*(1/d)^beta))

        if random.random() < exploration: # (exploitation)
            probs.sort(key=lambda x: x[1], reverse=True)
            next_city = probs[0][0]
        else: # (exploration)
            probs = [(c, p / total_pheromone) for c, p in probs]
            next_city = random.choices([c for c, p in probs])[0]
            # weights = [p / total_pheromone for c, pin probs]
            # next_city = random.choices([c for c, p in probs], weights=weights)[0]

        self.v_cities.append(next_city)
        self.total_distance += self.distances[c_city][next_city]
        # print(self.total_distance)

    def update_edges_pheromone(self, pheromone):
        #Emphasis on the best route  
        for i in range(len(self.v_cities) - 1):
            c_city = self.v_cities[i]
            next_city = self.v_cities[i+1]
            pheromone[c_city][next_city] += 1.0 / self.total_distance
        #Evaporating 
        for i in range(len(pheromone)):
            for j in range(len(pheromone[i])):
                pheromone[i][j] *= (1.0 - evaporation)
              
        return pheromone

    def reset_ant(self):
        self.v_cities = [self.start_city]
        self.total_distance = 0.0


In [4]:
def run(num_of_ants, num_iterations, distances, pheromone, cities):
    # start_cities= [random.randint(0,len(cities)-1) for i in range(num_ants)]
    # ants = [Ant(start_cities[i], distances, cities) for i in range(num_ants)]
    
    start_city = 0
    ants = [Ant(start_city, distances, cities) for i in range(num_of_ants)]
    for iteration in range(num_iterations):
        pheromone_map=[]
        for ant in ants:
            while len(ant.v_cities) < len(distances):
                ant.select_next_city(pheromone)
            ant.total_distance += distances[ant.v_cities[-1]][ant.v_cities[0]]  # Return back to the first city
            ant.v_cities.append(start_city)

        for ant in ants:
            pheromone = ant.update_edges_pheromone(pheromone)

        if(iteration%10==0):
            for ant in ants:   
              pheromone_ant=[]
              for i in range(len(ant.v_cities) - 1):
                    c_city = ant.v_cities[i]
                    next_city = ant.v_cities[i+1]
                    pheromone_ant.append(pheromone[c_city][next_city])
              pheromone_map.append(pheromone_ant)
            best_ant_index, best_ant= min(enumerate(ants), key=lambda x: x[1].total_distance)
            print("Iteration {}:".format(iteration))
            print("Best Distance = {}".format(best_ant.total_distance))
            print("Pheromone Map = {}".format(pheromone_map))
            print("Pheromone Best Ant = {} \n".format(pheromone_map[best_ant_index]))
            print("Visited {}".format(best_ant.v_cities))


        for ant in ants:
            #print("Total distance {}".format(ant.total_distance))
            ant.reset_ant()



# 10 Cities

In [5]:
distances1, pheromone1, cities1 = generate_cities(num_cities=10, min_distance=3, max_distance=40)
distances1

[[0, 12, 19, 14, 14, 14, 28, 35, 13, 24],
 [12, 0, 27, 24, 13, 20, 25, 32, 11, 18],
 [19, 27, 0, 7, 17, 7, 23, 28, 18, 25],
 [14, 24, 7, 0, 17, 7, 27, 32, 18, 26],
 [14, 13, 17, 17, 0, 10, 13, 21, 2, 10],
 [14, 20, 7, 7, 10, 0, 19, 25, 11, 19],
 [28, 25, 23, 27, 13, 19, 0, 7, 15, 7],
 [35, 32, 28, 32, 21, 25, 7, 0, 22, 14],
 [13, 11, 18, 18, 2, 11, 15, 22, 0, 10],
 [24, 18, 25, 26, 10, 19, 7, 14, 10, 0]]

In [6]:
cities1

[(4, 16),
 (3, 28),
 (22, 8),
 (15, 7),
 (16, 24),
 (18, 14),
 (28, 31),
 (35, 33),
 (14, 25),
 (21, 33)]

In [7]:
print("testing on 10 cities and 5 ants")
run(5, 51, distances1, pheromone1, cities1)

testing on 10 cities and 5 ants
Iteration 0:
Best Distance = 124.0
Pheromone Map = [[0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.6198176396244583, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742], [0.5953862686567165, 0.6198176396244583, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165, 0.5953862686567165], [0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.6198176396244583, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742], [0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.6198176396244583, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742], [0.614921370967742, 0.614921370967742, 0.614921370967742, 0.614921370967742, 0.6198176396244583, 0.614921370967742, 0.614921370967742, 

In [8]:
print("testing on 10 cities and 1 ants")
run(1, 51, distances1, pheromone1, cities1)

testing on 10 cities and 1 ants
Iteration 0:
Best Distance = 124.0
Pheromone Map = [[0.07114043219616961, 0.07173563212066497, 0.06539296427684656, 0.06429852769712767, 0.06389414809948737, 0.061855393810837316, 0.05699626917123877, 0.05623948178294544, 0.055794941069698026, 0.04573289851030775]]
Pheromone Best Ant = [0.07114043219616961, 0.07173563212066497, 0.06539296427684656, 0.06429852769712767, 0.06389414809948737, 0.061855393810837316, 0.05699626917123877, 0.05623948178294544, 0.055794941069698026, 0.04573289851030775] 

Visited [0, 1, 8, 4, 5, 2, 3, 9, 6, 7, 0]
Iteration 10:
Best Distance = 124.0
Pheromone Map = [[0.07243198322813293, 0.06108368165519967, 0.05887213012534465, 0.0511939452502579, 0.06084828441724438, 0.059399152913366664, 0.04864780513824404, 0.05681540586410242, 0.06033961102472333, 0.041596151843664435]]
Pheromone Best Ant = [0.07243198322813293, 0.06108368165519967, 0.05887213012534465, 0.0511939452502579, 0.06084828441724438, 0.059399152913366664, 0.04864780

In [9]:
print("testing on 10 cities and 10 ants")
run(10, 51, distances1, pheromone1, cities1)

testing on 10 cities and 10 ants
Iteration 0:
Best Distance = 109.0
Pheromone Map = [[0.06914171173150653, 0.00212818885918798, 0.05112724833115922, 0.04629087348135758, 0.05812460460906088, 0.0021005065168907457, 0.0689835054634565, 0.05721074269063711, 0.010722439917581667, 0.0455872590220493], [0.06914171173150653, 0.0668944307721779, 0.0689835054634565, 0.05721074269063711, 0.049649508494730016, 0.05112724833115922, 0.04629087348135758, 0.05812460460906088, 0.056405672690779174, 0.0455872590220493], [0.06914171173150653, 0.0668944307721779, 0.0689835054634565, 0.05721074269063711, 0.049649508494730016, 0.05112724833115922, 0.04629087348135758, 0.05812460460906088, 0.056405672690779174, 0.0455872590220493], [0.06914171173150653, 0.0668944307721779, 0.0689835054634565, 0.00438941034844905, 0.05812460460906088, 0.056405672690779174, 0.01007778189025905, 0.0189345460279508, 0.011908308795028551, 0.004389413722074479], [0.06914171173150653, 0.0668944307721779, 0.0689835054634565, 0.0572

In [10]:
print("testing on 10 cities and 20 ants")
run(20, 51, distances1, pheromone1, cities1)

testing on 10 cities and 20 ants
Iteration 0:
Best Distance = 114.0
Pheromone Map = [[0.001066483299278854, 0.0665259742700586, 0.05328653692986042, 0.048314414506349, 0.06697058905177182, 0.05762968949863875, 0.001088939492380728, 0.003438059508514389, 0.0013244205315390302, 0.001324416369507177], [0.06997493224465315, 0.06553286571034488, 0.06283364948097979, 0.054507977093763586, 0.0665259742700586, 0.05328653692986042, 0.048314414506349, 0.06697058905177182, 0.05762968949863875, 0.04522776371864116], [0.06997493224465315, 0.0010357681069013901, 0.00905582986606991, 0.013223174768175397, 0.0665259742700586, 0.005568570515628278, 0.017966246116093786, 0.003951929672289044, 0.003438059508514389, 0.0049022309689207], [0.06997493224465315, 0.0013131639688958759, 0.013223174768175397, 0.0665259742700586, 0.005568570515628278, 0.017966246116093786, 0.06697058905177182, 0.0015694089745754724, 0.003438059508514389, 0.0049022309689207], [0.06997493224465315, 0.06553286571034488, 0.0628336494

# **20 Cities**

In [11]:
distances2, pheromone2, cities2 = generate_cities(num_cities=20, min_distance=3, max_distance=40)
distances2

[[0, 27, 39, 37, 16, 9, 27, 17, 32, 26, 28, 5, 35, 7, 23, 12, 29, 12, 19, 29],
 [27, 0, 15, 11, 11, 17, 27, 12, 18, 14, 11, 21, 15, 21, 5, 26, 26, 16, 9, 28],
 [39,
  15,
  0,
  16,
  23,
  30,
  28,
  27,
  14,
  28,
  25,
  34,
  26,
  35,
  20,
  34,
  25,
  27,
  20,
  28],
 [37,
  11,
  16,
  0,
  22,
  27,
  37,
  20,
  26,
  15,
  12,
  31,
  10,
  30,
  14,
  37,
  36,
  27,
  20,
  38],
 [16, 11, 23, 22, 0, 7, 21, 8, 19, 16, 16, 11, 23, 12, 8, 15, 21, 5, 2, 22],
 [9, 17, 30, 27, 7, 0, 23, 8, 25, 18, 19, 4, 26, 5, 13, 12, 25, 4, 9, 25],
 [27, 27, 28, 37, 21, 23, 0, 29, 14, 37, 36, 25, 42, 28, 27, 15, 3, 19, 20, 2],
 [17, 12, 27, 20, 8, 8, 29, 0, 27, 9, 11, 12, 18, 10, 7, 21, 30, 10, 10, 31],
 [32,
  18,
  14,
  26,
  19,
  25,
  14,
  27,
  0,
  32,
  30,
  28,
  34,
  30,
  21,
  24,
  11,
  21,
  17,
  14],
 [26, 14, 28, 15, 16, 18, 37, 9, 32, 0, 3, 21, 9, 19, 10, 30, 37, 20, 17, 39],
 [28, 11, 25, 12, 16, 19, 36, 11, 30, 3, 0, 23, 7, 21, 9, 31, 36, 20, 16, 38],
 [5, 21, 34, 

In [12]:
cities2

[(4, 9),
 (29, 20),
 (35, 34),
 (40, 18),
 (18, 18),
 (13, 13),
 (7, 36),
 (21, 10),
 (21, 37),
 (30, 6),
 (32, 9),
 (9, 11),
 (39, 8),
 (11, 8),
 (26, 16),
 (3, 21),
 (10, 38),
 (13, 17),
 (20, 20),
 (7, 38)]

In [13]:
print("testing on 20 cities and 20 ants")
run(20, 51, distances2, pheromone2, cities2)

testing on 20 cities and 20 ants
Iteration 0:
Best Distance = 177.0
Pheromone Map = [[0.1562956038526903, 0.1570578982825823, 0.14855547838219482, 0.15004809769228655, 0.14607313529920718, 0.15069923706348276, 0.1467196726268234, 0.15641166717925625, 0.14788289936182136, 0.14562154124827684, 0.1445721632393504, 0.1496271778967072, 0.14566242018745612, 0.13488064803694985, 0.14183005006623498, 0.15910746099554746, 0.14525258527750262, 0.13696863021041272, 0.14434107520402117, 0.14270562959536126], [0.1562956038526903, 0.1570578982825823, 0.14855547838219482, 0.12427244840686055, 0.13009763192455018, 0.12836807617008586, 0.12836807617008586, 0.1235906153285986, 0.12555621902722872, 0.12555621902722872, 0.12555621902722872, 0.15069923706348276, 0.12555621902722872, 0.1223068447082304, 0.14183005006623498, 0.15910746099554746, 0.14525258527750262, 0.13696863021041272, 0.14434107520402117, 0.14270562959536126], [0.1562956038526903, 0.1570578982825823, 0.14855547838219482, 0.1500480976922865

In [14]:
print("testing on 20 cities and 10 ants")
run(10, 51, distances2, pheromone2, cities2)

testing on 20 cities and 10 ants
Iteration 0:
Best Distance = 159.0
Pheromone Map = [[0.04460544469728756, 0.0407883891594386, 0.0018148027197707368, 0.035304644809868814, 0.01221759406988591, 0.004240829204562125, 0.03444759207822286, 0.013664610911697142, 0.03865631526501676, 0.028190094684905773, 0.025849445173899727, 0.028792549620736883, 0.02803828130198491, 0.02597962872081309, 0.004038580512952081, 0.010323635462005938, 0.01454178889580427, 0.0037238177261849866, 0.00832027123777596, 0.008277196966519108], [0.04460544469728756, 0.0407883891594386, 0.036233631852189904, 0.03537073300497772, 0.021534643657514445, 0.01955232516684841, 0.016268253065535504, 0.0295813959736171, 0.03865631526501676, 0.028190094684905773, 0.025849445173899727, 0.028792549620736883, 0.02803828130198491, 0.007974745202718259, 0.02733716804026705, 0.03637392666106608, 0.035304644809868814, 0.02418885168957704, 0.03444759207822286, 0.0077704952596465435], [0.04460544469728756, 0.0407883891594386, 0.0362336

In [15]:
print("testing on 20 cities and 5 ants")
run(5, 51, distances2, pheromone2, cities2)

testing on 20 cities and 5 ants
Iteration 0:
Best Distance = 182.0
Pheromone Map = [[0.040429508067410615, 0.03142138397240345, 0.022732580123725542, 0.03288125105645192, 0.029350321675821182, 0.031140307583016995, 0.019983178934917044, 0.030255540406936853, 0.0352928463346724, 0.03967815965419681, 0.0026375525227135322, 0.03444485699395615, 0.037726568020940036, 0.03321118915050815, 0.03838718670365728, 0.006697668816277914, 0.003217125068704998, 0.0027091702227413795, 0.002703483215943727, 0.005109886752706987], [0.040429508067410615, 0.002421036056009793, 0.0024210332103321037, 0.014871380304390261, 0.030255540406936853, 0.0352928463346724, 0.03967815965419681, 0.03460076238843434, 0.003046714483290482, 0.002421033281582136, 0.03838718670365728, 0.005412840984659456, 0.008828419395821558, 0.03288125105645192, 0.002503791258793678, 0.0024210876570524504, 0.03444485699395615, 0.037726568020940036, 0.0026612521751235674, 0.0025309640370179327], [0.040429508067410615, 0.0052436077809244

In [16]:
print("testing on 20 cities and 5 ants")
run(1, 51, distances2, pheromone2, cities2)

testing on 20 cities and 5 ants
Iteration 0:
Best Distance = 199.0
Pheromone Map = [[0.043316142930597545, 0.04219815847912546, 0.04220095619556282, 0.03822762911229429, 0.004522613065326644, 0.03878757992623085, 0.03390219231966731, 0.033002414464247835, 0.020419867381249922, 0.01131173501339213, 0.03368354467505871, 0.007827028012338984, 0.037934278579044446, 0.006529518713393013, 0.02018929935410091, 0.035841232696056435, 0.028252011052825276, 0.023825449648495457, 0.028132894757138448, 0.012169005012279162]]
Pheromone Best Ant = [0.043316142930597545, 0.04219815847912546, 0.04220095619556282, 0.03822762911229429, 0.004522613065326644, 0.03878757992623085, 0.03390219231966731, 0.033002414464247835, 0.020419867381249922, 0.01131173501339213, 0.03368354467505871, 0.007827028012338984, 0.037934278579044446, 0.006529518713393013, 0.02018929935410091, 0.035841232696056435, 0.028252011052825276, 0.023825449648495457, 0.028132894757138448, 0.012169005012279162] 

Visited [0, 11, 13, 5, 17,